In [490]:
import os
import bs4
import regex
import shutil
import random
import zipfile
from bs4 import BeautifulSoup

In [2]:
# from txt_to_dataset import find_characters
# from txt_to_dataset import find_author
# from txt_to_dataset import find_end

---

In [3]:
TXT_DIR = "theatregratuit-txt"
off = "\t"
if os.path.isdir(TXT_DIR):
    fnames = [x for x in os.listdir(TXT_DIR) if ".txt" == os.path.splitext(x)[-1]]
    n_files = len(fnames)

In [614]:
tmp_dir = "tmp"
if not os.path.isdir(tmp_dir):
    os.mkdir(tmp_dir)

---

In [5]:
def printlines(lines):
    print('\n'.join(lines))

In [6]:
def printsep(s):
    print(s*40)

In [7]:
def underprint(x):
    print(x)
    print('-'*len(x))

In [119]:
def print_index_split(lines, index):
    start = index - 10
    if start < 0: start = 0
    printlines(lines[start:index])
    printsep('*')
    printlines(lines[index:index+10])

In [9]:
def print_file(f, whole_file=False):
    underprint(f)
    print()
    raw, lines, lines_len = get_lines(f)
    if whole_file:
        printlines(lines)
    else:
        lines_shortened, _ = shortened_lines(f, lines, lines_len)
        printlines(lines_shortened)

In [496]:
def print_random_file(files, whole_file=False):
    total = len(files)
    ind = random.randint(0, total)
    f = files[ind]
    print(f"file {ind}/{total}")
    print_file(f, whole_file=whole_file)

---

In [10]:
def get_lines(input_name):
    with open(os.path.join(TXT_DIR, input_name), "r", encoding="utf-8", errors="ignore") as f:
        raw = f.read()
        lines = [l.strip() for l in raw.split('\n')]
    return raw, lines, len(lines)

In [11]:
def shortened_lines(fname, lines, lines_len):
    author_index = find_author(fname, lines, lines_len) 
    if not author_index:
        author_index = index_of_regex_match(lines, REGICES["additional_author"])
    char_index = find_characters(fname, lines, lines_len)        
    end_index = find_end(fname, lines, lines_len)
    # print(fname, char_index, author_index, end_index)
    start_index = max(author_index, char_index)
    return lines[start_index:end_index], {
        "char_index": char_index, 
        "author_index": author_index, 
        "end_index": end_index,
        "start_index": start_index
    }

In [12]:
def end_of_block_and_trim(lines, lines_len, j, k):
    # find end of block
    while j+k < lines_len and not regex.match(REGICES["blank_line"], lines[j + k]):
        k += 1
    # trim empty lines
    while j+k < lines_len and regex.match(REGICES["blank_line"], lines[j + k]):
        k += 1     
    return k

In [368]:
def find_characters(fname, lines, lines_len):  
    feydeau = False
    if fname in (
        "count-2047890-LE_ROI.txt",
        "count-3031455-Nono.txt",
        "count-1712853-LE_VERITABLE_SAINT_GENEST.txt",
        "count-3263112-LA_MORT_DE_WALLENSTEIN.txt",
        "count-3397657-LA_VEINE.txt",
        "count-2158361-ANDROMEDE.txt",
        "count-2037607-LHabit_vert.txt",    
    ):
        feydeau = True
    vega = False
    char_index = 0
    found_char = False
    limit = 15
    
    if fname == "count-1537913-LES_ESPAGNOLS_EN_DANEMARK.txt":
        limit = 26
    for j, l in enumerate(lines[:limit]):
        if found_char:
            break
        if "Feydeau" in l and fname not in (
            "count-2199212-Les_Paves_de_lours.txt",
            "count-2316496-On_va_faire_la_cocotte.txt",            
        ):
            feydeau = True
        if "Lope de Vega" in l:
            vega = True
        if regex.match(REGICES["character"], l):
            found_char = True
            k = 1
            # files with space after char > k + 1
            if fname in (
                "count-2158361-ANDROMEDE.txt",
                "count-2037607-LHabit_vert.txt",
                "count-1182649-LE_MONDE_OU_L.txt",
                "count-1037209-LES_MAMELLES_DE_TIRESIAS_-_Guillaume_Apollinaire.txt",
                "count-958980-Le_gendre_de_Monsieur_Poirier_-_Emile_Augier.txt"                
            ):
                k += 1
            # more annoying exceptions
            if fname in ("count-959005-LOrphelin_de_la_Chine_-_Voltaire.txt",):
                # find init caps words only, no space (lest we go to the end)
                while j+k < lines_len and regex.match(REGICES["WORD"], lines[j + k]):
                    k += 1    
            elif vega: 
                # all vega can go up to "La scène"
                while j+k < lines_len and not regex.match(REGICES["la_scene"], lines[j + k]):
                    k += 1                
            else:
                k = end_of_block_and_trim(lines, lines_len, j, k)
                # annoying exceptions: more chars after blank line
                if feydeau:
                    k = end_of_block_and_trim(lines, lines_len, j, k)
                    # some files like Feydeay
                    if fname in ("count-3263112-LA_MORT_DE_WALLENSTEIN.txt",
                                 "count-3397657-LA_VEINE.txt"):
                        k = end_of_block_and_trim(lines, lines_len, j, k)
            char_index = j + k          
    return char_index

In [14]:
def find_author(fname, lines, lines_len):
    author_index = 0
    found_author = False
    for j, l in enumerate(lines[:10]):
        if found_author:
            break
        if regex.match(REGICES["author"], lines[j]):
            found_author = True
            # annoying exceptions
            if fname in ("count-1049195-Arret_36_de_lautobus_40_-_Jean_Sibil.txt",
                         "count-1396213-Roberto_Succo.txt"):  
                k = 1
            elif fname == "count-1541294-Olaf_loriginal.txt":
                k = 2 
            else:
                k = 2  # we assume "de\n\nauthor"
                while j + k < lines_len and not regex.match(
                    REGICES["blank_line"], lines[j + k]
                ):
                    k += 1
            author_index = j + k + 1
    return author_index

In [15]:
def find_end(fname, lines, lines_len):
    end_index = lines_len - 1
    found_end = False
    for j, l in enumerate(reversed(lines[-4:])):
        if found_end:
            break
        if regex.search(REGICES["fin"], l):
            found_end = True
            end_index = end_index - j - 1
    return end_index

In [16]:
def index_of_regex_match(lines, r, trim=True):
    lines_len = len(lines)
    ind = 0
    found = False
    for i,l in enumerate(lines):
        if found: break
        if regex.search(r, l):
            found = True
            k = 1            
            if trim: 
                while i + k < lines_len and \
                 regex.match(REGICES["blank_line"], lines[i + k]):
                    k += 1
            ind = i + k
            break
    return ind 

In [17]:
def file_data(f, verbose=True):
    if verbose:
        underprint(f)
        print()
    raw, lines, lines_len = get_lines(f)
    lines_shortened, indices = shortened_lines(f, lines, lines_len)
    lines_shortened_len = len(lines_shortened)
    data = {
        "fname": f,
        "raw": raw,
        "lines": lines,
        "lines_len": lines_len,
        "lines_shortened": lines_shortened,
        "lines_shortened_len": lines_shortened_len
    }
    data.update(indices)
    return data

In [608]:
def make_regices():
    return {
        "blank_line": regex.compile("^\s*$"),
        "blank_line_with_rubbish": regex.compile("^[\p{Z}\p{P}]*$"),
        
        # trim beginning & end of files
        # -----------------------------
        "character": regex.compile(
            "^((les)*pe*rsonna *ge|(les)*acteurs|dramatis personae|avertissement|entreparleur|biographies|apparences|personrage|persongueules|pépersonâge)",
            regex.IGNORECASE,
            ),
        "author": regex.compile("^de$",regex.IGNORECASE),
        "additional_author": regex.compile("^(\(Auteur inconnu\)|" +\
                                           "Voltaire|" +\
                                           "de\s+Georges Courteline|" +\
                                           "Translation en prose de Jean Sibil|" +\
                                           "Anton Pavlovitch Tchekhov|" +\
                                           "EUGENE LABICHE, A. LEFRANC et MARC-MICHEL|" +\
                                           "Alexandre Hardy)$"),
        "la_scene": regex.compile("^La scène"),        
        "fin": regex.compile("(fin|rideau|f1n|inachev|manque)", regex.IGNORECASE),
        
        
        # search for lines made of full caps + punct/space, as well as some
        # very common words listing characters, and didascalia in ()
        "caps_full": regex.compile("^\s*((\p{Lu}+|seule?|puis|puis tout le monde|moins|et)\s*[.’'<>;,]*\s*)+\s*(\(.*?\))*\s*$"),
        # common annoying starts "M.", "L'"
        "annoying_init": regex.compile("^[\p{P}\p{Z}]*(M\.|\p{Lu}')\p{Z}*"),
        # words (with possible - or '), and space, with punctuation at the end
        "caps_init": regex.compile("^\s*(([\p{Lu}\p{Pc}\p{Pd}1]+\p{Zs}*)+)(\s*[\p{Po}]\s*)"),
        "char_lc_dot_dash": regex.compile("^(\p{Z}*M*[\p{Ll}\p{Z}]+\.)\p{Z}*\p{Pd}+\p{Z}*(.*)$"),
        "trailing_space": regex.compile("\s*$"),
        "trailing_punct": regex.compile("\s*[.,:]*\s+$"),
        "non_breaking_space": regex.compile(" {2,}"), # non-breaking space
        "M." : regex.compile("M.\s"),
        "dot": regex.compile("\s*\.\s*"),
        "final_dot": regex.compile("\s*\.\s*$"),
        "final_comma": regex.compile(",$"),
        
        # include rare errors like ".-—Blah" or ". -— Blah"
        "dash": regex.compile("[,.](\s*[-–—]+|-[–—])\s*"),
        "dash_and_more": regex.compile("(.*?)[.;:,]*\s*-*[-–—]\s*"),
        "colon": regex.compile("\s*:\s*"),
        "colon_and_more": regex.compile("(.*?)\s*:\s*"),
        "didasc_and_more": regex.compile(".*?(?<!M)[.:]\s*"),
        # "char_no_dot": regex.compile("^\s*([A-Z1'-]+)\s*$"),
        # # using all 3 lengths: -, –, —, bc of inconsistencies
        # "char_dash": regex.compile("^\s*([A-Z1'-]{2,})\.[\s-]+[–—]\s"),
        # "char_comma_dash": regex.compile("^\s*([A-Z1'-]{2,},.*?)\.[\s-]+[–—]\s"),
        # "char_dot_more": regex.compile("^\s*([A-Z1'-]{2,}\.)(.*)$"),
        # "char_no_dot_more": regex.compile("^\s*([A-Z1'-]{2,})(.*)$"),
        
        # regices for corpus splitting
        # ----------------------------
        "WORD": regex.compile("\p{Lu}{2,}"), 
        "WORD_INIT": regex.compile("^((L')*\p{Lu}{2,}|[AB]\.)"),         
        "CHAR_ONELINE": regex.compile("^\p{Lu}+[\p{Z}\p{P}]*$"),
        "CHAR_ONELINE_DOT": regex.compile("^\p{Lu}+\p{Z}*\.\p{Z}*$"),
        "CHAR_ONELINE_COLON": regex.compile("^\p{Lu}+\p{Z}*:\p{Z}*$"),
        "INIT_char_DOT_n_DASH" : regex.compile("^\p{L}+\.\p{Z}+\p{Pd}"),
        "INIT_char_COLON" : regex.compile("^\p{L}+\p{Z}+:"),          
        "INIT_CHAR_DOT_N_DASH" : regex.compile("^\p{Lu}+\.\p{Z}+\p{Pd}"),
        "INIT_CHAR_COLON" : regex.compile("^\p{Lu}+\p{Z}+:"),      
        "init_char_dot_n_dash" : regex.compile("^\p{Ll}+\.\p{Z}+\p{Pd}"),
        "init_char_colon" : regex.compile("^\p{Ll}+\p{Z}+:"),
        
        
        # regices for innards cleaning
        # ----------------------------
        # letters & space, possibly a parenthesis, : or — at the end
        "char_colon_or_dash": regex.compile("^((?:L'|M\.)*[\p{L}\p{Pd}\p{Z}]+)" +\
                                            "(\p{Z}\(.*?\))*" +\
                                            "\p{Z}[:—]\p{Z}"),
        
        # lower case character, optionally with didasc, ending in dot and dash, 
        "char_lc_dot_dash": regex.compile("^([\p{Ll}\p{Z}]+)" +\
                                          "(([,\p{Z}].*?)*)" + \
                                          "\.\p{Z}*\p{Pd}\p{Z}*"),
        
        # char (with numbers & spaces) followed or not by :
        "char_oneline_colon": regex.compile("^((L')*[\p{L}\p{N}\p{Z}]+?)\p{Z}*:$"),
        
        "char_colon": regex.compile("^(\p{L}{2,})\p{Z}*:\p{Z}*"),
        # "char_colon_more": regex.compile("^\s*([A-Z1'-]{2,})\s*:\s*(.*)$"),
        # "first_scene": regex.compile("(sc[èe]ne (premi[èe]re|I)|premier tableau|I, 1\.)", regex.IGNORECASE),
        # "act_or_scene": regex.compile("(acte|sc[èe]ne|^I\s*$)", regex.IGNORECASE),
        
        # block (multiline) regices (to act on raw text)
        # "characters_block": regex.compile("\n*\s*PERSONNAGES?(.*?)(\n\s*\n)+", regex.DOTALL),
        
        # line cleanup
        "sc_ene": regex.compile("SC\p{Z}+[EÈ]NE", regex.IGNORECASE),
        "deuxi_eme": regex.compile("DEUXI\p{Z}ÈME", regex.IGNORECASE)
    }

REGICES = make_regices()

---
## All files.

In [20]:
fnames_it = iter(fnames)
ind = 0

In [31]:
fname = next(fnames_it)
# fname = "count-1396213-Roberto_Succo.txt"
ind +=1 
underprint(fname)
print(ind)
print()
raw, lines, lines_len = get_lines(fname)
# printlines(lines)
lines_shortened, data = shortened_lines(fname, lines, lines_len)
# printlines(lines_shortened)
print_index_split(lines, max(data["char_index"], data["author_index"]))

count-1046024-La_Revolte_-_Adam.txt
-----------------------------------
4

LA REVOLTE

de

Villiers de L’Isle-Adam

Personnages :
ELISABETH. 27 ans
FELIX, 40 ans

****************************************
Le salon d’un banquier : Ameublement ronge, noir et or. Porte au fond, lustre, tapis.  A droite, une causeuse près de la cheminée. Un peu de feu. A gauche, bien en scène, une table-bureau, chargée de livres de caisse, de papiers. Lampe et son abat-jour éclairant la table. Le reste du théâtre est un peu dans l’ombre. Une horloge, au-dessus de la porte au fond. marque bientôt minuit. La salle est très profonde. Au lever du rideau. .Mme Elisabeth est assise près de la table. Elle est accoudée et pensive, elle est vêtue très simplement, en noir: M. Félix. en face d'elle, compulse des lettres et des billets de banque.

Scène première
ELISABETH, FELIX
FELIX : (après un grand silence) Quelle heure ?
ELISABETH : Très tard.
FELIX : (très froidement} Déjà minuit ? (II remonte la lampe en clignan

---

## The meaty part: reformatting

---

In [609]:
def split_by_regex(fnames, r, threshold=1, orig_file=False):
    with_r = []
    without_r = []
    for fname in fnames:
        found = 0
        raw, lines, lines_len = get_lines(fname)
        if not orig_file:
            lines, _ = shortened_lines(fname, lines, lines_len)
        for i, l in enumerate(lines):
            if regex.search(r, l):
                found += 1
                if found == threshold:
                    break
        if found == threshold:
            with_r.append(fname)
        else:
            without_r.append(fname)
    print("with:", len(with_r))
    print("without:", len(without_r))
    return with_r, without_r

In [22]:
def add_markers(lines):
    lines.append("<|e|>")
    lines.append("<|s|>")
    return lines

In [23]:
def remove_trailing_lines(lines):
    if regex.match(REGICES["blank_line"], lines[-1]):
        for i, l in enumerate(reversed(lines)):
            if not regex.match(REGICES["blank_line"], l):
                return lines[:-i]
    else:
        return lines

In [24]:
def markers_final_cleanup(lines):
    lines = ["<|s|>"] + lines   
    lines = remove_trailing_lines(lines)
    lines.append("<|e|>") 
    return lines

---

### Splitting dataset into similar files

In [610]:
caps, no_caps = split_by_regex(fnames, REGICES["WORD_INIT"])

with: 1012
without: 71


In [607]:
"count-2843262-La_grande_voix_de_la_presse.txt" in caps

True

In [369]:
caps_w, no_caps_w = split_by_regex(fnames, REGICES["WORD"])

with: 1038
without: 45


In [586]:
test, no_test = split_by_regex(fnames, REGICES["char_oneline_colon"], threshold=20)

with: 56
without: 1027


In [597]:
print_random_file(test)

file 39/56
count-2683966-MARINO_FALIERO.txt
--------------------------------

La scène est à Venise, en 1355.

ACTE PREMIER.
SCÈNE I.
(L'appartement du doge.)
ELENA. (Elle est assise et brode une écharpe.)
Une écharpe de deuil, sans chiffre, sans devise!
Hélas ! triste présent ! mais je l'avais promise,
Je devais l'achever... Vaincu par ses remords,
Du moins après ma faute, il a quitté nos bords;
Il recevra ce prix de l'exil qu'il s'impose.
(Elle se lève et s'approche de la fenêtre.)
Le beau jour ! que la mer où mon œil se repose,
Que le ciel radieux brillent d'un éclat pur,
Et que Venise est belle entre leur double azur!
Lui seul ne verra plus nos lagunes chéries :
Il n'est qu'une Venise ! on n'a pas deux patries !...
Je pleure... Oui, Fernando, sur mon crime et le tien.
Pourquoi pleurer? j'ai tort : les pleurs n'effacent rien.
Mon bon, mon noble époux aime à me voir sourire;
Eh bien, soyons heureuse, il le faut...
Je veux lire.
Le Dante, mon poète ! essayons... je ne puis.
Nous le li

In [ ]:
test, no_test = split_by_regex(fnames, REGICES["INIT_char_COLON"])

In [262]:
char_oneline, no_char_oneline = split_by_regex(fnames, REGICES["CHAR_ONELINE"])

with: 681
without: 402


In [126]:
char_oneline_dot, char_oneline_nodot = split_by_regex(char_oneline, REGICES["CHAR_ONELINE_DOT"])

with: 432
without: 249


In [127]:
char_oneline_colon, char_oneline_nocolon = split_by_regex(char_oneline, REGICES["CHAR_ONELINE_COLON"])

with: 69
without: 612


In [112]:
init_char_dot_n_dash, no_init_char_dot_n_dash = split_by_regex(fnames, REGICES["INIT_CHAR_DOT_N_DASH"])

with: 392
without: 691


In [82]:
fnames_it = iter(with_it)
# fnames_it = iter(without)

In [111]:
fname = next(fnames_it)
print_file(fname)

count-2877388-ROCAMBOLLE_LE_BATELEUR.txt
----------------------------------------

La scène se passe à Bayonne, en 1803.

ACTE I
Le théâtre représente une place publique. A gauche, un cabaret avec deux tables en plein vent. A droite, le derrière de la baraque de ROCAMBOLLE, espèce de tente fermée avec des rideaux mobiles; on lit au-dessus de la porte :
ENTRÉE PABTICULIÈRE DES ACTEURS. LE PUBLIC N'ENTRE PAS ICI.
SCÈNE PREMIÈRE.
MIC-MAC, CHALAMEL à la table placée au premier plan, SOLDATS, à la deuxième table, deuxième plan.
CHŒUR.
Enfants d'la giberne,
Noyons dans l'cognac
Les chagrins d'caserne,
L'ennui du bivouac.
Honte à c'lui qui bouge
Et qui r'cule d'un ch'veu
D'vant un habit rouge,
Ou d'vant un vin bleu.
Enfants, etc., etc.
MIC-MAC, qui a de grands cheveux blonds, trinquant avec CHALAMEL. — A vot’santé, militaire et la société.
CHALAMEL. — A la vôtre et à celle du premier Consul. (Tous se lèvent et trinquent. — Se rasseyant.) Qu'est-ce que vous cachez donc dans ce sac-là, monsieur

In [114]:
init_char_colon2, no_init_char_colon2 = split_by_regex(init_char_dot_n_dash, REGICES["INIT_CHAR_COLON"])

with: 112
without: 280


In [113]:
init_char_colon, no_init_char_colon = split_by_regex(no_init_char_dot_n_dash, REGICES["INIT_CHAR_COLON"])

with: 88
without: 603


In [118]:
with_it, without = split_by_regex(fnames, REGICES["init_char_dot_n_dash"])

with: 14
without: 1069


In [117]:
with_it, without = split_by_regex(fnames, REGICES["init_char_colon"])

with: 8
without: 1075


In [2105]:
# fname = next(fnames_it)
data = file_data(fname)
lines_formatted = []
done_first = False
for i, l in enumerate(data["lines_shortened"]):
    
    colon_re = regex.search(REGICES["char_colon_or_dash"], l)
    if colon_re:
        if done_first:
            lines_formatted = add_markers(lines_formatted)
        done_first = True
        start = colon_re.group(1).upper()
        if colon_re.group(2):
            start += colon_re.group(2)
        start += "."
        end = l[colon_re.span()[1]:]       
        lines_formatted.append(start)
        if end: 
            lines_formatted.append(end)
        continue
        
#     char_lc_dot_dash = regex.search(REGICES["char_lc_dot_dash"], l)
#     if char_lc_dot_dash:
#         if done_first:
#             lines_formatted = add_markers(lines_formatted)
#         done_first = True
#         start = char_lc_dot_dash.group(1).upper()
        
#         if char_lc_dot_dash.group(2):
#             start += char_lc_dot_dash.group(2)    
#         start += "."
#         end = l[char_lc_dot_dash.span()[1]:]       
#         lines_formatted.append(start)
#         if end: 
#             lines_formatted.append(end)
#         continue    
        
#     char_oneline_colon = regex.search(REGICES["char_oneline_colon"], l)
#     if char_oneline_colon:
#         if done_first:
#             lines_formatted = add_markers(lines_formatted)
#         done_first = True
#         start = char_oneline_colon.group(1).upper()    
#         start += "." 
#         end = l[char_oneline_colon.span()[1]:]               
#         lines_formatted.append(start)
#         if end: 
#             lines_formatted.append(end)
#         continue         
        
    lines_formatted.append(l)
lines_formatted = markers_final_cleanup(lines_formatted)

printlines(lines_formatted)

count-3121428-Don_Diegue_avait_raison.txt
-----------------------------------------

<|s|>
A droite (pour le spectateur) deux immeubles en arc concave ; devant, une allée bitumée étroite ; sur la gauche, un square avec trois bancs visibles, dont un presque en face de l'entrée de l'immeuble le plus près de la rampe, quelques arbres, un toboggan pour petits enfants. Vers le fond, à gauche, extrémité d'un immeuble concave lui aussi, qui ferme presque le square, laissant juste place à l'allée ; sur le devant, une autre allée, qui permet d'accéder au square également par la gauche et par la droite.

1. (Un homme en short, assez costaud sans être une force de la nature, un paquet sous un bras, appuie et appuie sur une sonnette de l'entrée au premier plan.)
L'HOMME.
Ouvrez ! Ou-vrez ! C'est le livreur Damercot. (Il appuie de toutes ses forces sur toutes les sonnettes.) Ouvrez ! Ou-vrez ! Y a personne dans c't'immeuble ! Ça marche pas. Les sonnettes, aucune ne marche. Saleté. Saleté.
(Il fait 

---
Some files have nothing in caps after trimming.

In [1868]:
with_it, without = split_by_regex(fnames, r"^\w+\s*:")

with: 454
without: 629


In [1879]:
fnames_it = iter(with_it)
# fnames_it = iter(without)

In [1880]:
fname = next(fnames_it)
print_file(fname)

count-2454059-Champignol_malgre_lui.txt
---------------------------------------

Le 1er acte à Paris dans l'hôtel de CHAMPIGNOL.
Le 2* acte à Clermont.
Le 3* acte chez Mme Rivolet, dans les environs de Clermont.

ACTE I
A droite la chambre de madame CHAMPIGNOL. — A gauche, premier plan, porte donnant sur les appartements. — Au deuxième plan, grande fenêtre, porte au fond, donnant sur l'antichambre. — Au fond de l'antichambre, porte donnant sur l'escalier. — Tableaux sur des chevalets. — Etudes sur les murs, etc. - A droite, premier plan, une table flanquée de deux chaises, sur la table une tasse de chocolat servie; à gauche, un canapé, et, à côté du canapé et à droite, deux chaises volantes. — Au fond contre le mur, une toile placée sur une chaise et retournée.
SCENE PREMIERE
SAINT-FLORIMOND, puis ANGELE.
Au lever du rideau, la scène est vide. Un coucou, sonne huit heures, puis on entend une clef tourner dans la serrure de la porte d'entrée, au fond de l'antichambre, qui s'ouvre et SAI

Le dash following chars. 

In [1811]:
with_dash, without_dash = split_by_regex(fnames, REGICES["dot_n_dash"])

with: 777
without: 306


In [1812]:
# fnames_it = iter(caps_word)
fnames_it = iter(no_caps_word)

In [1852]:
fname = next(fnames_it)
print_file(fname)

count-2488208-Une_passade_de_Donata.txt
---------------------------------------

Un grand salon avec un escalier, les volets sont baissés (fermés). Un couple en tenue de ville, Xavier et Irène Gevrais, elle la trentaine, lui la quarantaine; un homme en tenue plus décontractée, la trentaine, Robert.

1. Xavier : En haut ?
Robert : Elle va descendre.
Irène : Elle va d'abord nous faire attendre. Jusqu'au moment où on sera exaspérés. Je connais ses raffinements.
Robert : Elle a changé.
Xavier (railleur) : C'est une grande actrice; Donata change souvent.
Robert : Elle a renoncé à son art.
Irène : Foutaise.
Xavier : Vous nous avez bien annoncés comme je vous avais dit ?
Robert : Oui oui. Je ne connais pas votre nom de toute façon. "L'homme à l'impeccable complet."
Irène : Et mon chapeau ? Vous lui avez parlé de mon nouveau chapeau ? (Il est très élégant et sobre; rien de comique.)
Robert : A une époque où l'on n'en porte plus guère, je n'aurais eu garde d'oublier. Curieusement cela a semblé 

---
## All the trimming...

---
### Ending

Unproblematic (**hopefully**). `find_end` does the job, used in `shortened_lines`.

In [179]:
with_end, without_end = split_by_regex(fnames, REGICES["fin"])

with: 1083
without: 0


---
### Author

Split by formatted author: clean author files with formatted author name and files without.

In [352]:
auth_fname, no_auth_fnames = split_by_regex(fnames, REGICES["author"], orig_file=True)

with: 1070
without: 13


In [353]:
fnames_it = iter(no_auth_fnames)

In [364]:
fname = next(fnames_it)
raw, lines, lines_len = get_lines(fname)
author_index = index_of_regex_match(lines, REGICES["additional_author"], trim=True)
print_index_split(lines, author_index)


****************************************
Ruy Blas

1858

Personnages

RUY BLAS
DON SALLUSTE DE BAZAN
DON CÉSAR DE BAZAN
DON GURITAN


In [365]:
fnames_it = iter(auth_fname)

In [381]:
fname = next(fnames_it)
raw, lines, lines_len = get_lines(fname)
author_index = find_author(fname, lines, lines_len)
print_index_split(lines, author_index)

La Punaise

de

Vladimir Maïakovski

****************************************
Traduction de M. Wassiltchikov

PERSONNAGES :
PRISSIPKINE, PIERRE SKRIPKINE, ex-ouvrier, ex-membre du Parti; à présent, fiancé.
ZOIA BEREZKINE, ouvrière.
Famille RENAISSANCE :
ELZEVIRE DAVIDOVNA, fiancée, manucure, caissière du salon de coiffure.
ROSALIE PAVLOVNA, mère-coiffeuse.
DAVID OSSIPOVITCH, père-coiffeur.
OLEG BAYAN, autodidacte, ancien propriétaire.


Updating `find_author` & `shortened_lines` accordingly.

---
### Characters
After that, clean characters.

In [382]:
char, no_char = split_by_regex(fnames, REGICES["character"])

with: 889
without: 194


In [525]:
fnames_it = iter(char)
# fnames_it = iter(no_char)

In [534]:
fname = next(fnames_it)
print_file(fname, whole_file=True)

count-1355599-LE_CARROSSE_DU_SAINT.txt
--------------------------------------

LE CARROSSE DU SAINT-SACREMENT

Saynète

de

Prosper Mérimée

Tu veras que mis finezas
Te desenojan.
CALDERÔN
(Cual es la major perfection)

PERSONNAGES :
DON ANCRES DE RIBERA, vice-roi du Pérou .
L'EVEQUE DE LIMA.
LE LICENCIE TOMAS D‘ESQUIVEL.
MARTINEZ, secrétaire intime du vice-roi.
BALTHASAR, valet de chambre du vice-roi.
CAMILA PERICHOLE, comédienne.

La scène est à Lima, en 17...

Le cabinet du Vice-Roi.

Le VICE-ROI, en robe de chambre, assis dans un grand fauteuil auprès d'une table couverte de papiers. Une de ses jambes enveloppée de flanelle repose sur un coussin. Martinez debout auprès de la table, une plume à la main.

MARTINEZ : MM. les Auditeurs attendent la réponse de Votre Altesse.
LE VICE-ROI, d'un ton chagrin : Quelle heure est-il ?
MARTINEZ : Bientôt dix heures. Votre Altesse a justement le temps de s'habiller pour la cérémonie.
LE VICE-ROI : Le temps est beau, dis-tu ?
MARTINEZ : Oui, mons

Updating `find_character` & `shortened_lines`.

In [584]:
fnames_it = iter(fnames)

In [770]:
fname = next(fnames_it)
# fname = "count-959005-LOrphelin_de_la_Chine_-_Voltaire.txt"
# fname = "count-1049195-Arret_36_de_lautobus_40_-_Jean_Sibil.txt"
fname = "count-1541294-Olaf_loriginal.txt"
raw, lines, lines_len = get_lines(fname)
lines_shortened, indices = shortened_lines(fname, lines, lines_len)
print_index_split(lines, indices["author_index"] + indices["char_index"])

Olaf l'original

de

Jean Sibil
****************************************
(Catherine I)
Un salon-salle à manger au mobilier à la fois en harmonie et hétéroclite. Superbes rideaux. Il règne un certain laisser-aller : revues diverses en vrac dans un coin, coussins mal placés sur le divan, fauteuils mal placés (deux sont vis-à-vis pour étendre ses jambes sur l'autre)... Aux murs, très soigneusement accrochés, des photos de mode, des affiches de concerts exceptionnels (toutes musiques), des posters de films récents... Effet de surcharge.
Olaf (la cinquantaine sportive, tenue de footing, une petite queue de cheval et une mèche bleue sur le devant que, debout vers la cheminée, il examine dans une glace de poche) : Je sentais que l'orange me siérait mieux.
Corinne (la trentaine, avachie dans ses fauteuils vis-à-vis, en robe de chambre à grosses fleurs) : Chéri, je t'aime de toutes les couleurs.
Olaf : Tu t'en tires toujours comme ça au lieu de donner des conseils utiles.
Corinne (agressive) : 

---

### Space after "PERSONNAGES :"

In [1619]:
space_after_char = []
for f in fnames:
    raw, lines, lines_len = get_lines(f)
    if regex.search(r"\n\s*PERSONNAGES?\s*:*\n\s*\n", raw):
        space_after_char.append(f)
print(space_after_char)

['count-2158361-ANDROMEDE.txt', 'count-2037607-LHabit_vert.txt', 'count-1182649-LE_MONDE_OU_L.txt', 'count-1037209-LES_MAMELLES_DE_TIRESIAS_-_Guillaume_Apollinaire.txt', 'count-958980-Le_gendre_de_Monsieur_Poirier_-_Emile_Augier.txt']


---

### Checks for results with very few lines

In [1789]:
too_short = []
for f in fnames:
    data = file_data(f, verbose=False)
    dl = data["lines_shortened_len"]
    if dl > 10 and dl < 20:
        too_short.append(data)
len(too_short)

16

In [1790]:
too_short_it = iter(too_short)

In [1807]:
dn = next(too_short_it)
underprint(dn["fname"])
printlines(dn["lines_shortened"])
# printsep('-')
# printlines(dn["lines"])

StopIteration: 

---

# remove lines:
Traduction de

Collaborateur

# avertissement:

count-3141023-Ce_soir_on_improvise.txt

# list of chars without title: 

count-3013840-Menage_moderne.txt

count-2843262-La_grande_voix_de_la_presse.txt